In [ ]:
from shapely.geometry import box, Point, LineString
import pyrosm
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree
from scipy import stats
import geopandas
import pandas as pd
import numpy as np
import random
import networkx as nx
import math
from rasterio.crs import CRS
from sys import getsizeof

In [ ]:
pd.options.display.max_columns = 50
OUT_CRS = CRS.from_epsg(27700)  # https://epsg.io/27700

In [ ]:
# convenience function for printing
def print_size(obj, suffix='B'):
    num = getsizeof(obj)
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    print("%.1f%s%s" % (num, 'Yi', suffix))

In [ ]:
%%time
import pickle
from pathlib import Path

class Person():
    def __init__(self, uid, household_uid, age, pos):
        self.uid = uid
        self.household_uid = household_uid
        self.age = age
        self.pos = pos
    
# households_gdf = pickle.loads(Path('pickles/london/households_gdf').read_bytes())
people = pickle.loads(Path('../pickles/tower_hamlets/people_list').read_bytes())
trimmed_work = pickle.loads(Path('../pickles/tower_hamlets/workplaces_gdf').read_bytes())
boundary = pickle.loads(Path('../pickles/tower_hamlets/boundary').read_bytes())

In [ ]:
trimmed_work = trimmed_work.to_crs(OUT_CRS)
trimmed_work.plot(markersize=0.1)

In [ ]:
%%time
# get all public transport
full_edges = pd.read_csv('../../data/uk_aggregate/Data_Release_v1.11/edges.csv')
full_nodes = pd.read_csv('../../data/uk_aggregate/Data_Release_v1.11/nodes.csv')
full_nodes = geopandas.GeoDataFrame(full_nodes, geometry=geopandas.points_from_xy(full_nodes.lon, full_nodes.lat), crs="EPSG:4326")
layers = pd.read_csv('../../data/uk_aggregate/Data_Release_v1.11/layers.csv')

# Clip the nodes to the boundary
nodes = geopandas.clip(full_nodes, boundary)
nodes_set = set(nodes.set_index(['node', 'layer']).index)

In [ ]:
%%time
# Get only the relevant time schedules
from collections import namedtuple 

Ride = namedtuple('Ride', ['start_time', 'duration'])
event_records = []

with Path('../../data/uk_aggregate/Data_Release_v1.11/events.txt').open() as f:
    for line in f:
        eles = line.strip().split(',')
        (ori_node, des_node, ori_layer, des_layer) = [int(ele.strip()) for ele in eles[:4]]
        
        if (not ((ori_node, ori_layer) in nodes_set and (des_node, des_layer) in nodes_set)):
            continue
        
        iter_times = iter(eles[4:])
        rides = [Ride(int(start_time), int(duration)) for (start_time, duration) in zip(iter_times, iter_times)]
        
        event_records.append((ori_node, des_node, ori_layer, des_layer, rides))

events = pd.DataFrame.from_records(event_records, columns=['ori_node', 'des_node', 'ori_layer', 'des_layer', 'rides'])
del event_records

In [ ]:
# We can't do anything with nodes that don't have schedules
nodes_with_schedules = set(events.set_index(['ori_node', 'ori_layer']).index).union(events.set_index(['des_node', 'des_layer']).index)
nodes_set = nodes_set.intersection(nodes_with_schedules)

In [ ]:
# Only select edges that start or end at the clipped nodes
des_edges = full_edges[full_edges.set_index(['des_node', 'des_layer']).index.isin(nodes_set)]
ori_edges = full_edges[full_edges.set_index(['ori_node', 'ori_layer']).index.isin(nodes_set)]
unclipped_edges = des_edges.merge(ori_edges) # inner merge to get edges that start and end in the boundary

In [ ]:
# We can't do anything with nodes that don't have edges
nodes_with_edges = set(unclipped_edges.set_index(['ori_node', 'ori_layer']).index).union(unclipped_edges.set_index(['des_node', 'des_layer']).index)
nodes_set = nodes_set.intersection(nodes_with_schedules)

In [ ]:
# refilter nodes to remove ones without necessary information
nodes = nodes[nodes.set_index(['node', 'layer']).index.isin(nodes_set)]

In [ ]:
%%time
# Draw lines for the edges
old_index = full_nodes.index
full_nodes = full_nodes.set_index(['node', 'layer'])
unclipped_edges = geopandas.GeoDataFrame(unclipped_edges, geometry=unclipped_edges.apply(lambda x: LineString([
    full_nodes.loc[(x.ori_node, x.ori_layer)].geometry, 
    full_nodes.loc[(x.des_node, x.des_layer)].geometry
]), axis=1))
full_nodes = full_nodes.reset_index()
full_nodes.index = old_index

# commented for now as we require des AND ori nodes to be within the boundary instead of OR
# Clip those edges to the boundary too
# edges = geopandas.clip(unclipped_edges.reset_index(), boundary)  
# Get rid of empty geometries
#edges = edges[~edges.is_empty]
edges = unclipped_edges

In [ ]:
fig, ax = plt.subplots(figsize=(17, 17))
nodes.plot(ax=ax, markersize=1)
edges.plot(ax=ax, linewidth=0.1)
del fig, ax

In [ ]:
nodes['osmid'] = nodes['node'].astype(str) + '_' + nodes['layer'].astype(str)
nodes['x'] = nodes.geometry.x
nodes['y'] = nodes.geometry.y
nodes.set_index('osmid', verify_integrity=True, inplace=True, drop=False) # keep the column as osmnx needs it (as well as it being the index of the DF)
nodes

In [ ]:
edges['u'] = edges['ori_node'].astype(str) + '_' + edges['ori_layer'].astype(str)
edges['v'] = edges['des_node'].astype(str) + '_' + edges['des_layer'].astype(str)
edges['key'] = edges.reset_index()['index']
edges['osmid'] = edges['u'].astype(str) + '_' + edges['v'].astype(str)
edges.set_index(['u', 'v', 'key'], inplace=True)
edges = edges.set_crs("EPSG:4326")
edges

In [ ]:
import osmnx
graph = osmnx.utils_graph.graph_from_gdfs(nodes, edges, None)

In [ ]:
fig, ax = osmnx.plot_graph(graph)

In [ ]:
osmnx.utils_graph.graph_to_gdfs(graph, nodes=False)

In [ ]:
graph_node_ids = osmnx.utils_graph.graph_to_gdfs(graph, edges=False).osmid # the graph module removes some nodes (probably unconnected ones, TODO investigate)

In [ ]:
%%time
# transit_node_positions = [tuple(coord) for coord in nodes.to_crs(OUT_CRS)[['x', 'y']].to_numpy()]
transit_node_positions = np.array(list(nodes.loc[graph_node_ids].to_crs(OUT_CRS).geometry.centroid.apply(lambda x: (x.x, x.y))))
nodeKdTree = cKDTree(data=transit_node_positions)

In [ ]:
# %%time
# amount = 10_000
# print(f'For {amount} iterations')
# for index, person in enumerate(people):
#     if index > amount:
#         break
#     osmnx.distance.get_nearest_node(graph, person.pos)

In [ ]:
# %%time
# print(f'For {len(people)} iterations')
# for index, person in enumerate(people):
#     nodeKdTree.query(x=person.pos, k=1)

In [ ]:
# inspired by https://gis.stackexchange.com/a/301935
def cKDQueryRadius(gdA_in, gdB_in, radius=300, reproject=True, p=2.0, workers=1):
    gdA = gdA_in.copy()
    gdB = gdB_in.copy()
    if reproject:
        in_crs = gdA.crs
        gdA = gdA.to_crs(OUT_CRS)
        gdB = gdB.to_crs(OUT_CRS)
        
    nA = np.array(list(gdA.geometry.centroid.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.centroid.apply(lambda x: (x.x, x.y))))
    
    btree = cKDTree(nB)
    elements_in_radius = btree.query_ball_point(nA, r=radius, p=p, workers=workers)

    gdf = pd.concat(
        [gdA.reset_index(drop=True),
        pd.Series(elements_in_radius, name='Elements in Radius')], axis=1
    )
    if reproject:
        gdf = gdf.to_crs(gdA_in.crs)
    return gdf

In [ ]:
# %%time
# work_nearest_nodes = cKDQueryRadius(trimmed_work, nodes, radius=1_500, workers=5)
# print_size(work_nearest_nodes)

In [ ]:
# %%time
# nodes_nearest_households = cKDQueryRadius(nodes, households_gdf, radius=1_500, workers=5)
# print_size(nodes_nearest_households)

In [ ]:
# %%time
# household_positions = list(households_gdf.geometry.centroid.apply(lambda x: (x.x, x.y)))
# households_kd_tree = cKDTree(data=household_positions)
# print_size(households_kd_tree) doesn't work

In [ ]:
# %%time
# workplace_positions = [(x, y) for x, y in zip(trimmed_work.geometry.x, trimmed_work.geometry.y)]

In [ ]:
# %%time
# for pos in workplace_positions:
#     households_kd_tree.query_ball_point(pos, 500)

In [ ]:
# households_gdf

In [ ]:
# TODO Remove when Person pickle is remade for South London
# for person in people:
#     person.pos = (person.pos.x, person.pos.y)

In [ ]:
people_df = pd.DataFrame(data=[{'uid': person.uid, 'x': person.pos[0], 'y': person.pos[1], 'age': person.age} for person in people])
print_size(people_df)
del people

In [ ]:
# people_gdf = geopandas.GeoDataFrame(people_df, geometry=geopandas.points_from_xy(people_df.x, people_df.y), crs=OUT_CRS)
# fig, ax = plt.subplots(figsize=(10,10))
# people_gdf.plot(ax=ax, marker_size=0.1)
# del fig, ax, test, people_gdf

In [ ]:
projected_boundary = boundary.copy()
projected_boundary = projected_boundary.set_crs("EPSG:4326")
projected_boundary = projected_boundary.to_crs(OUT_CRS)
bounds = projected_boundary.bounds
boundary_minx = bounds.loc[bounds.index[0], 'minx']
boundary_maxx = bounds.loc[bounds.index[0], 'maxx']
boundary_miny = bounds.loc[bounds.index[0], 'miny']
boundary_maxy = bounds.loc[bounds.index[0], 'maxy']

In [ ]:
bucket_size = 250 # meters for OUT_CRS
y_len = int(math.ceil((boundary_maxy - boundary_miny) / bucket_size))
x_len = int(math.ceil((boundary_maxx - boundary_minx) / bucket_size))

print(f'y: {y_len}, x: {x_len}')

In [ ]:
unemployed = people_df.loc[(17 <= people_df['age']) & (people_df['age'] <= 67)].copy()
print(len(unemployed))

In [ ]:
unemployed['bucket_x'] = np.ceil(((unemployed['x'] - boundary_minx) / bucket_size)).astype(int) - 1
unemployed['bucket_y'] = np.ceil(y_len - ((unemployed['y'] - boundary_miny) / bucket_size)).astype(int) - 1

In [ ]:
# unemployed_bucket = np.array([[]] * (y_len * x_len) + [[1]], dtype=object)[:-1].reshape((y_len, x_len))

In [ ]:
def make_buckets():
    unemployed_bucket = [[[] for x in range(x_len)] for y in range(y_len)]
    for person in unemployed.itertuples():
        unemployed_bucket[person.bucket_y][person.bucket_x].append(person.uid)
    
    return unemployed_bucket

In [ ]:
%%time
unemployed_bucket = make_buckets()

In [ ]:
len(unemployed_bucket)

In [ ]:
# https://stackoverflow.com/questions/44865023/how-can-i-create-a-circular-mask-for-a-numpy-array
def create_circular_mask(h, w, centre_y=None, centre_x=None, radius=None):
    if centre_x is None: # use the middle of the image
        centre_x = int(w / 2)
    if centre_y is None:
        centre_y = int(h / 2)
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(centre_y, centre_x, (w - centre_x), (h - centre_y))

    y, x = np.ogrid[-centre_y:(h - centre_y), -centre_x:(w - centre_x)]
    mask = (x * x) + (y * y) <= (radius * radius)
    return mask

In [ ]:
%%time
counts = np.array([list(map(len, row)) for row in unemployed_bucket])

In [ ]:
%%time
mask = create_circular_mask(h=y_len, w=x_len)

In [ ]:
indices = np.argwhere(mask)
indices.shape

In [ ]:
%%time
probabilities = counts[mask] / np.sum(counts[mask])
probabilities.size

In [ ]:
%%timeit
chosen_buckets = np.random.choice(np.arange(indices.shape[0]), 1_000, p=probabilities)

In [ ]:
# sanity check the sampling matches the underlying probabilities
chosen_buckets = np.random.choice(np.arange(indices.shape[0]), 50_000, p=probabilities)

test = np.zeros(counts.shape)
for index, chosen in enumerate(chosen_buckets):
    test[tuple(indices[chosen])] += 1
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(test, origin='lower', vmax=10)
del fig, ax, test, chosen_buckets

In [ ]:
class NoViablePeopleError(Exception):
    pass

In [ ]:
def valid_unemployed_within_dist(y, x, dist, cache_size=200):
    counts = np.array([list(map(len, row)) for row in unemployed_bucket])
    y = y_len - int(math.ceil((y - boundary_miny) / bucket_size)) - 1
    x = int(math.ceil((x - boundary_minx) / bucket_size)) - 1
    dist = dist / bucket_size
    mask = create_circular_mask(h=y_len, w=x_len, centre_y=y, centre_x=x, radius=dist)
    
    indices = np.argwhere(mask)
    if np.sum(counts[mask]) == 0:
        raise NoViablePeopleError(f'No viable unemployed people left around (x:{x}, y:{y})')
    probabilities = counts[mask] / np.sum(counts[mask])
#     print(f'x: {x}, y: {y}, indices: {indices}, probabilities: {probabilities}')
    
    chosen_buckets = np.random.choice(np.arange(indices.shape[0]), cache_size, p=probabilities) # a cache of sampled choices
    chosen_iter = iter(chosen_buckets)
    failures = 0
    
    while True:
        try:
            next_bucket_pos = tuple(indices[next(chosen_iter)])
        except StopIteration: # used up the cache, repopulate with some more random choices
            chosen_buckets = np.random.choice(np.arange(indices.shape[0]), cache_size, p=probabilities)
            chosen_iter = iter(chosen_buckets)
            continue
        try: 
            next_person = unemployed_bucket[next_bucket_pos[0]][next_bucket_pos[1]].pop()
        except IndexError: # no unemployed people in that bucket
            failures += 1
            if failures > 300: # we had a lot of failures, recalc the probability map
                failures = 0
                
                counts = np.array([list(map(len, row)) for row in unemployed_bucket])
                
                total_unemployed_left = np.sum(counts[mask])
                if total_unemployed_left == 0:
                    raise NoViablePeopleError(f'No viable unemployed people left around ({x}, {y})')
                
                probabilities = counts[mask] / total_unemployed_left
                
                chosen_buckets = np.random.choice(np.arange(indices.shape[0]), cache_size, p=probabilities)
                chosen_iter = iter(chosen_buckets)
            continue
            
        yield next_person

In [ ]:
unemployed_bucket = make_buckets()

# pick a random workplace to query around
rand_workplace = trimmed_work.loc[[len(trimmed_work) // 3]]

(y, x) = (rand_workplace.geometry.y, rand_workplace.geometry.x)
# (y, x) = (397000, 385000)
print(f'({float(x)}, {float(y)})')
valid_unemployed_gen_5k = valid_unemployed_within_dist(y, x, 3_000)

# sanity check the sampling matches the underlying probabilities
chosen_people = [(next(valid_unemployed_gen_5k)) for i in range(45_000)]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(15, 15))
ax1.set_aspect('equal')
ax2.set_aspect('equal')
# sanity check
chosen = people_df.loc[chosen_people]
chosen.plot(ax=ax1, x='x', y='y', kind='scatter', s=0.1)
(xmin, xmax, ymin, ymax) = (chosen['x'].min(), chosen['x'].max(), chosen['y'].min(), chosen['y'].max())
people_df[(people_df['x'] > xmin) & (people_df['x'] < xmax) & (people_df['y'] > ymin) & (people_df['y'] < ymax)].plot(ax=ax2, x='x', y='y', kind='scatter', s=0.1)

ax1.plot(x, y, 'r+')
ax2.plot(x, y, 'r+')
# rand_workplace.plot(ax=ax1, markersize=100, c='red', marker='x')
# rand_workplace.plot(ax=ax2, markersize=100, c='red', marker='x')
del fig, ax1, ax2, chosen_people, chosen, xmin, ymin, xmax, ymax, rand_workplace

In [ ]:
unemployed_bucket = make_buckets()
valid_unemployed_gen_5k = valid_unemployed_within_dist(y, x, 5_000)
%timeit -r 10 -n 15_000 next(valid_unemployed_gen_5k)

In [ ]:
del valid_unemployed_gen_5k, y, x

In [ ]:
# households within 60k
# generate a probability heatmap
# ravel and sample it
# if there are no free slots set it to 0
# if 100 misses in a row, regenerate probability map

In [ ]:
# pick a random vector
# query-ball around that vector
# try X vectors, then new dist

In [ ]:
%%time
# uses about 5-6GB of memory for Manchester with 4400 nodes and 6200 edges 
# about 12-14GB of mem for London with 7000 nodes 16000 edges
travel_times = dict(nx.all_pairs_dijkstra_path_length(graph, cutoff=90, weight='minutes'))  # {node(str): {target(str): dist(num)}}

In [ ]:
import pickle
from pathlib import Path
Path('../pickles/london/travel_times').write_bytes(pickle.dumps(travel_times))

In [ ]:
from copy import deepcopy
MIN_TRAVEL_TIME = 15
graph_node_ids_to_index = {node_id: index for index, node_id in enumerate(list(graph_node_ids))}

transit_nodes_to_commuting_nodes = {graph_node_ids_to_index[src_node_id]: [target_node_id for target_node_id, time in times_dict.items() if time > MIN_TRAVEL_TIME] 
                   for src_node_id, times_dict in travel_times.items()}

def get_transit_nodes_to_commuting_nodes():
    return deepcopy(transit_nodes_to_commuting_nodes)
    
del travel_times, graph_node_ids_to_index

In [ ]:
class NoNearbyTransitNodesError(Exception):
    pass

In [ ]:
def get_nearby_transit_nodes(workplace):
    dist = 1_000
    nearby_transit_nodes = nodeKdTree.query_ball_point((workplace.geometry.x, workplace.geometry.y), dist)
    while len(nearby_transit_nodes) == 0 and dist <= 5_000:
        dist += 1000
        nearby_transit_nodes = nodeKdTree.query_ball_point((workplace.geometry.x, workplace.geometry.y), dist)
    if len(nearby_transit_nodes) == 0:
        raise NoNearbyTransitNodesError(f"No transit nodes were found within walking distance of workplace {workplace.index}")
    
    return nearby_transit_nodes

In [ ]:
nodes_to_radius_search = dict(zip(list(graph_node_ids), list(map(lambda pos: valid_unemployed_within_dist(pos[1], pos[0], 5_000, 100), transit_node_positions))))

In [ ]:
from enum import Enum
class TransportType(Enum):
    PUBLIC_TRANSIT = 0,
    DRIVING = 1,
    CYCLING = 2,
    WALKING = 3

In [ ]:
%%time
to_allocate = len(unemployed)
print(f'Trying to allocate {to_allocate} people, {len(trimmed_work)} workplaces')

unemployed_bucket = make_buckets()
reachable_nodes = get_transit_nodes_to_commuting_nodes()

people_to_workplaces = [None] * len(people_df)
people_to_transport_type = [None] * len(people_df)

failures = 0
successes = 0
workplace_indices = []

for index, workplace in enumerate(trimmed_work.sample(frac=1).itertuples()): # iterate over shuffled workplaces
    if successes >= to_allocate:
        print(f'Allocated all people after {index} workplaces')
        break
    if index < 9:
        workplace_indices.append(workplace.index)
        
    transport_options = [TransportType.PUBLIC_TRANSIT, TransportType.DRIVING, TransportType.CYCLING, TransportType.WALKING]
    
    try:
        nearby_transit_nodes = get_nearby_transit_nodes(workplace)
    except NoNearbyTransitNodesError:
        transport_options.remove(TransportType.PUBLIC_TRANSIT)
        
    valid_unemployed_gen_60k = valid_unemployed_within_dist(workplace.geometry.y, workplace.geometry.x, 60_000)
    valid_unemployed_gen_20k = valid_unemployed_within_dist(workplace.geometry.y, workplace.geometry.x, 20_000)
    valid_unemployed_gen_5k = valid_unemployed_within_dist(workplace.geometry.y, workplace.geometry.x, 5_000)
    
    for _ in range(workplace.capacity):
        # TODO update to better reflect real distributions if possible, maybe weight it by workplace size, more likely to walk if you own the business
        random.shuffle(transport_options)
        random_transport = iter(transport_options)
        
        person_id = None
        transport_type = None
        
        while (person_id == None):
            try:
                transport_type = next(random_transport)
            except StopIteration:
                break
            
            if transport_type == TransportType.PUBLIC_TRANSIT:
                source_node_index = np.random.choice(nearby_transit_nodes)
                try:
                    while (person_id == None):
                        dest_osmid = random.choice(reachable_nodes[source_node_index])
                        try:
                            person_id = next(nodes_to_radius_search[dest_osmid])
                        except StopIteration:
                            reachable_nodes[source_node_index].remove(dest_osmid)
                except (NoViablePeopleError, IndexError):
                    transport_options.remove(transport_type)
            elif transport_type == TransportType.DRIVING:
                try:
                    person_id = next(valid_unemployed_gen_60k)
                except NoViablePeopleError:
                    transport_options.remove(transport_type)
            elif transport_type == TransportType.CYCLING:
                try:
                    person_id = next(valid_unemployed_gen_20k)
                except NoViablePeopleError:
                    transport_options.remove(transport_type)
            elif transport_type == TransportType.WALKING:
                try:
                    person_id = next(valid_unemployed_gen_5k)
                except NoViablePeopleError:
                    transport_options.remove(transport_type)
        
        if person_id:
            if transport_type:
                people_to_transport_type[person_id] = transport_type
            
            people_to_workplaces[person_id] = workplace.index
            successes += 1
            if successes >= to_allocate:
                break
        else:
            failures +=1
            continue            
    
    # workplace_row.capacity
    # workplace.index    
    
print(f'Successes: {successes}, Failures: {failures}')

In [ ]:
from collections import defaultdict
workplaces_to_people = defaultdict(list)
for person_id, workplace_index in enumerate(people_to_workplaces):
    if workplace_index is not None:
        workplaces_to_people[workplace_index].append(person_id)

example_workplaces_to_people = {ind: workplaces_to_people[ind] for ind in workplace_indices}

fig, ax = plt.subplots(figsize=(15, 15))
ax.set_aspect('equal')
marker_styles = [
                ('red', 'o'), ('blue', 'o'), ('green', 'o'),
                ('red', 'P'), ('blue', 'P'), ('green', 'P'),
                ('red', '*'), ('blue', '*'), ('green', '*')
                ]

for (workplace_index, worker_indices), (color, marker) in zip(example_workplaces_to_people.items(), marker_styles):
    trimmed_work.iloc[[workplace_index]].plot(ax=ax, markersize=100, c=color, marker=marker)
    workers = people_df.iloc[worker_indices]
    workers.plot(ax=ax, x='x', y='y', kind='scatter', s=30, c=color, marker=marker)
del fig, ax, workplaces_to_people